In [1]:
# Import modules
from bs4 import BeautifulSoup
import os
import pandas as pd
from movie_webscraping_utils import *

### 1. Scraping movie info from IMDb and WIKI pages

In this notebook, we scrape the data about the top rated movies from IMDB and Wikipedia. Later, in a separate Jupyter notebook, we will use this data to build different topic models so we can retrieve latent topics present in movies plots.

First, we access the IMDb page with 250 top rated movies and scrape the urls of each individual movie. Then, we will scrape the basic info from each movie page, such as movie title, genre and realese date, as well as the synopsis page url. We then proceed to the synopsis page and scrape a detailed storyline of each movie, which we will use later to build our topic models. Moreover, we also try to scrape the individual movie plots from the wikipedia pages dedicated to those movies. In the majority of movies, we can find the wiki webpage by simply adding a movie title to the wikipedia homepage. For the rest, we have to tweak the title slightly to get the correct webpage.  

In [2]:
url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250%20lang_eng'
# Make HTTP requests and get a content from a given url
html = read_request(url)
soup = BeautifulSoup(html, 'html.parser')

# Scraping web pages for each movie on 250 IMDB top chart list
movie_webpage_list = []
for a in soup.select( 'td.titleColumn a' ):
    url_new = a.get('href')
    url_new = convert_if_relative_url(url, url_new)
    if not is_absolute_url(url_new):
        print('URL not valid')
    movie_webpage_list.append(url_new)

# Check if we downloaded all 250 movie URLs
assert len(movie_webpage_list) == 250

In [3]:
# Getting English translated titles from the movies
headers = {'Accept-Language': 'en-US, en;q=0.5'}

wiki_url = 'http://en.wikipedia.org/wiki/'
error_log_list = []

count = 1
for movie_url in movie_webpage_list:
    # Getting movie information from IMDB page of each movie
    movie_html = read_request(movie_url, headers=headers)
    movie_soup = BeautifulSoup(movie_html, 'html.parser')
    
    # Scraping movie title, genre and release date
    try:
        title = movie_soup.find(
            'h1', {"data-testid":"hero-title-block__title"}).get_text(strip=True)
    except:
        error_log_list.append('Obtaining the movie title failed ' + movie_url)
        title = None
    try:
        genre = movie_soup.find('div', {"data-testid":"genres"}).get_text(" ")
        genre_list = list(genre.split())
    except:
        error_log_list.append('Obtaining the movie genre failed ' + movie_url)
        genre_list = None
    try:
        release_date = movie_soup.find(
            'li',{"role":"presentation"}).find('span').get_text(" ")
    except:
        error_log_list.append('Obtaining the movie release date failed ' + movie_url)
        release_date = None
    
   
    # Accessing the synopsis page of each movie and scraping the storyline from it
    try:
        synopsis_ulr = movie_soup.find(
            'ul', {"data-testid":"storyline-plot-links"}).find_all('a')[1].get('href')
        synopsis_ulr = convert_if_relative_url(movie_url, synopsis_ulr)
        synopsis_html = read_request(synopsis_ulr)
        synopsis_soup = BeautifulSoup(synopsis_html, 'html.parser')
        synopsis = synopsis_soup.find(
            'ul', {"id":"plot-synopsis-content"}).get_text(" ", strip=True)
        assert (synopsis is not None and synopsis is not "")
    except:
        error_log_list.append('Obtaining the movie synopsis failed ' + movie_url)
        synopsis = None

    # Getting the movie plot from the Wikipedia page
    try:
        movie_url_wiki = os.path.join(wiki_url, title.replace(' ', '_'))
        # Extracting the plot summary
        wiki_plot = extract_wiki_plot(movie_url_wiki)
        assert (wiki_plot is not None and  wiki_plot is not "")
    except:
        wiki_plot = None
        error_log_list.append(title + ' ' + release_date + ' ' + movie_url_wiki)

    # Adding the scraped movie data to 'top_250_movies.csv'
    update_csv_file('top_250_movies.csv',
                    [[count, title, genre_list, release_date, synopsis, wiki_plot]],
                    cols=['rank', 'title', 'genre', 'release_date', 'imdb_synopsis', 'wiki_plot'],
                    folder_path = './data/')
    
    count += 1

top_250_movies.csv file does not exist. Creating new file!
Read failed: http://en.wikipedia.org/wiki/Léon:_The_Professional
Read failed: http://en.wikipedia.org/wiki/WALL·E
Read failed: http://en.wikipedia.org/wiki/Capharnaüm
Read failed: http://en.wikipedia.org/wiki/Amélie
Read failed: http://en.wikipedia.org/wiki/Nausicaä_of_the_Valley_of_the_Wind


### 2. Inspecting the error logs

In the cell below, we look at the cases for which scraping the movie data failed. This happened for quite a few movies - 114 in total. When scraping IMDb website, we only encountred problems in the synopsis part of the webpage (with 16 movies in total). On the other hand, there were many more problems when scraping the wiki pages (98 in total). The large number of problems with the wiki pages is due to our often incorrect initial guess for the webpage link - we simply add the movie title to the Wikipedia homepage. So we will need a little bit more effort to scrape the rest of wiki movie plots.

In [4]:
# Inspecting the error logs
print('Total number of fails: ', len(error_log_list))
print()
print('Number of imdb failed synopsis scraping: ',
      len([i for i in error_log_list if i.startswith('Obtaining ')]))
print([i for i in error_log_list if i.startswith('Obtaining ')])
print()
print('Number of wiki failed plot scraping: ',
      len([i for i in error_log_list if not i.startswith('Obtaining ')]))
print([i for i in error_log_list if not i.startswith('Obtaining ')])

Total number of fails:  114

Number of imdb failed synopsis scraping:  16
['Obtaining the movie synopsis failed https://www.imdb.com/title/tt0027977/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt8267604/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt0057565/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt0091251/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt0012349/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt0050976/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt0476735/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt3011894/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt0077711/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt0053198/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt0116231/', 'Obtaining the movie synopsis failed https://www.imdb.com/title/tt0113247/', '

### 3. Scraping the movie plots from IMDb pages for the movies that are missing synopsis

On further inspection of error_logs_list, we can see that all of the errors, encountered when scraping from the IMDb website, are for the movies that do not have a synopsis yet added to the page. So, we scrape their storyline plots instead. The storyline plots are quite shorter than the synopsis, but still contain relevant information about the movie storyline.

In [5]:
# Loading the partially completed dataset
movies_df = pd.read_csv('./data/top_250_movies.csv')

# Webscraping plot summaries from IMDb for movies without synopsis
for k in [i for i in error_log_list if i.startswith('Obtaining ')]:
    movie_url = k.split()[-1]
    movie_html = read_request(movie_url, headers=headers)
    movie_soup = BeautifulSoup(movie_html, 'html.parser')
    title = movie_soup.find(
            'h1', {"data-testid":"hero-title-block__title"}).get_text(strip=True)
    plot = movie_soup.find(
        'div', {"data-testid":"storyline-plot-summary"}
        ).find('div').find('div').find(text=True, recursive=False)
    if plot is None or "":
        print('Plot scraping failed', movie_url)
        continue
    # Adding wiki plot for the particular movie to movies_df
    add_data_to_df_field(movies_df, movies_df['title'] == title, 
                         'imdb_synopsis', plot)

,rank,title,genre,release_date,imdb_synopsis,wiki_plot
36,37,Modern Times,"['Comedy', 'Drama', 'Family']",1936,"Chaplin's last 'silent' film, filled with soun...",NaN


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
76,77,Capharnaüm,['Drama'],2018,"Capernaüm (""Chaos"") tells the story of Zain (Z...",NaN


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
78,79,High and Low,"['Crime', 'Drama', 'Mystery']",1963,A wealthy businessman is told his son has been...,NaN


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
95,96,Come and See,"['Drama', 'Thriller', 'War']",1985,"The feature film directed by Elem Klimov, shot...","In 1943, two Belarusian boys dig in a sand-fi..."


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
103,104,The Kid,"['Comedy', 'Drama', 'Family']",1921,"The opening title reads: ""A comedy with a smil...",NaN


...

,rank,title,genre,release_date,imdb_synopsis,wiki_plot
221,222,La Haine,"['Crime', 'Drama']",1995,The film follows three young men and their tim...,La Haine opens with a montage of news footage ...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
231,232,A Silent Voice: The Movie,"['Animation', 'Drama']",2016,"The story revolves around Shôko Nishimiya, a g...",Japanese high school student Shoya Ishida inte...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
236,237,Nausicaä of the Valley of the Wind,"['Animation', 'Adventure', 'Fantasy']",1984,An animated fantasy-adventure. Set one thousan...,NaN


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
239,240,Raatchasan,"['Action', 'Crime', 'Mystery']",2018,"Circumstances force Arun, an aspiring film dir...",The film opens with two old men discovering a ...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
247,248,Drishyam,"['Crime', 'Drama', 'Thriller']",2013,Georgekutty (Mohanlal) is a cable TV network o...,The film begins with Georgekutty - an man accu...


In [6]:
# Checking that imdb_synopsis column doesn't contain NaN values
print(movies_df['imdb_synopsis'].isnull().any())

False


### 4. Scraping the rest of Wikipedia movie plots

As we saw when inspecting the error log, we were not able to scrape the Wikipedia movie plots of 98 movies. We used a simple way of forming the movie wiki page urls by concatenating the wikipedia homepage with the name of the film.
(Example. `http://en.wikipedia.org/wiki/The_Wolf_of_Wall_Street`)
On further inspection of the movies that were unsucessfully scraped, we can observe two common patterns for their respective wiki urls - the first have `_(film)` after the movie title, while the second have `_(XXXX_film)` where `XXXX` stands for the release year of the film. An example of the first pattern is `http://en.wikipedia.org/wiki/The_Dark_Knight_(film)`, while an example for the second pattern is `http://en.wikipedia.org/wiki/Gladiator_(2000_film)`.

In [7]:
wiki_error_log_list = []
for error_log in [i for i in error_log_list if not i.startswith('Obtaining ')]:
    wiki_url = error_log.split()[-1]
    title = ' '.join(error_log.split()[0:-2])
    wiki_ulr_modified = wiki_url + '_(film)'
    try:
        # Extracting the plot summary
        wiki_plot = extract_wiki_plot(wiki_ulr_modified)
        assert (wiki_plot is not None and  wiki_plot is not "")
        # Adding wiki plot for the particular movie to movies_df
        add_data_to_df_field(movies_df, movies_df['title'] == title, 
                             'wiki_plot', wiki_plot)
    except:
        wiki_error_log_list.append(error_log)

,rank,title,genre,release_date,imdb_synopsis,wiki_plot
3,4,The Dark Knight,"['Action', 'Crime', 'Drama']",2008,The movie begins with a gang of men with clown...,A gang of criminals rob a Gotham City mob bank...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
17,18,One Flew Over the Cuckoo's Nest,['Drama'],1975,"In 1963 Oregon, Randle Patrick McMurphy (Nicho...","In Oregon in 1963, Randle Patrick McMurphy is ..."


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
19,20,Se7en,"['Crime', 'Drama', 'Mystery']",1995,In an unidentified city of constant rain and u...,Soon-to-retire Detective Lieutenant William So...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
20,21,The Silence of the Lambs,"['Crime', 'Drama', 'Thriller']",1991,Promising FBI Academy student Clarice Starling...,"In 1990, Clarice Starling is pulled from her F..."


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
27,28,Interstellar,"['Adventure', 'Drama', 'Sci-Fi']",2014,A group of elderly people are giving interview...,"In 2067, crop blights and dust storms threaten..."


...

Read failed: http://en.wikipedia.org/wiki/Nausicaä_of_the_Valley_of_the_Wind_(film)


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
244,245,The Princess Bride,"['Adventure', 'Family', 'Fantasy']",1987,Fairy tale story-within-a-story with an all-st...,The film is an enactment of a book that a gran...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
245,246,Sunrise,"['Drama', 'Romance']",1927,"In the summertime, described as vacation time,...",A vacationing Woman from the City (Margaret Li...


Read failed: http://en.wikipedia.org/wiki/Hera_Pheri_(film)


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
249,250,"Paris, Texas",['Drama'],1984,The film opens with sweeping shots of the vast...,Travis Henderson walks alone through the West ...


In [8]:
print('Number of remaining movies without wiki_plot:', len(wiki_error_log_list))

Number of remaining movies without wiki_plot: 50


In [9]:
wiki_error_log_list1 = []
for error_log in wiki_error_log_list:
    wiki_url = error_log.split()[-1]
    title = ' '.join(error_log.split()[0:-2])
    release_date = error_log.split()[-2]
    wiki_ulr_modified = wiki_url + '_(' + release_date + '_film)'
    try:
        # Extracting the plot summary
        wiki_plot = extract_wiki_plot(wiki_ulr_modified)
        assert (wiki_plot is not None and  wiki_plot is not "")
        # Adding wiki plot for the particular movie to movies_df
        add_data_to_df_field(movies_df, movies_df['title'] == title, 
                             'wiki_plot', wiki_plot)
    except:
        wiki_error_log_list1.append(error_log)

,rank,title,genre,release_date,imdb_synopsis,wiki_plot
4,5,12 Angry Men,"['Crime', 'Drama']",1957,"In a New York City courthouse, an eighteen-yea...",In the overheated jury room of the New York Co...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
21,22,City of God,"['Crime', 'Drama']",2002,Taking place over the course of over two decad...,The film begins in medias res with an armed ga...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
29,30,Parasite,"['Comedy', 'Drama', 'Thriller']",2019,Ki-woo Kim (Choi Woo-Shik) is a young man livi...,"The Kim family—father Ki-taek, mother Chung-so..."


Read failed: http://en.wikipedia.org/wiki/Léon:_The_Professional_(1994_film)
Read failed: http://en.wikipedia.org/wiki/Hara-Kiri_(1962_film)


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
32,33,The Pianist,"['Biography', 'Drama', 'Music']",2002,"""The Pianist"" begins in Warsaw, Poland in Sept...","In September 1939, Władysław Szpilman, a Polis..."


...

Read failed: http://en.wikipedia.org/wiki/Gangs_of_Wasseypur_(2012_film)


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
227,228,Rebecca,"['Drama', 'Mystery', 'Romance']",1940,"The film begins with a female voiceover: ""Last...",An inexperienced young woman meets aristocrati...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
228,229,Rush,"['Action', 'Biography', 'Drama']",2013,The following is based on a true story. The fi...,James Hunt and Niki Lauda are exceptional raci...


Read failed: http://en.wikipedia.org/wiki/Nausicaä_of_the_Valley_of_the_Wind_(1984_film)
Read failed: http://en.wikipedia.org/wiki/The_Battle_of_Algiers_(1966_film)


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
246,247,Hera Pheri,"['Action', 'Comedy', 'Crime']",2000,"The plot revolves around an eccentric trio, co...",The film begins with Ghanshyam Tripathi (Sunie...


In [10]:
print('Number of remaining movies without wiki_plot:', len(wiki_error_log_list1))
print(wiki_error_log_list1)

Number of remaining movies without wiki_plot: 10
['Léon: The Professional 1994 http://en.wikipedia.org/wiki/Léon:_The_Professional', 'Hara-Kiri 1962 http://en.wikipedia.org/wiki/Hara-Kiri', 'WALL·E 2008 http://en.wikipedia.org/wiki/WALL·E', 'Hamilton 2020 http://en.wikipedia.org/wiki/Hamilton', 'Capharnaüm 2018 http://en.wikipedia.org/wiki/Capharnaüm', 'Pather Panchali 1955 http://en.wikipedia.org/wiki/Pather_Panchali', 'Amélie 2001 http://en.wikipedia.org/wiki/Amélie', 'Gangs of Wasseypur 2012 http://en.wikipedia.org/wiki/Gangs_of_Wasseypur', 'Nausicaä of the Valley of the Wind 1984 http://en.wikipedia.org/wiki/Nausicaä_of_the_Valley_of_the_Wind', 'The Battle of Algiers 1966 http://en.wikipedia.org/wiki/The_Battle_of_Algiers']


We are left with 10 movies for which we still need to scrape the movie plot. For 6 of them we need to provide their complete urls as a list (see `movie_url_wiki_list`) and their titles (`movie_title_wiki_list`), because they contain special letter characters. Nevertheless, we can still apply the same function (`extract_wiki_plot`) to scrape their storylines. For the last 4 (`Hamilton`, `Pather_Panchali`, `Gangs of Wasseypur` and `The Battle of Algiers`), we need to scrape indivdually because their HTML webpages are slightly different from the rest of the movies.

In [11]:
movie_url_wiki_list = ['https://en.wikipedia.org/wiki/L%C3%A9on:_The_Professional',
                       'https://en.wikipedia.org/wiki/Harakiri_(1962_film)',
                       'https://en.wikipedia.org/wiki/WALL-E',
                       'https://en.wikipedia.org/wiki/Capernaum_(film)',
                       'https://en.wikipedia.org/wiki/Am%C3%A9lie',
                       'https://en.wikipedia.org/wiki/Nausica%C3%A4_of_the_Valley_of_the_Wind_(film)']
movie_title_wiki_list = ['Léon: The Professional', 'Hara-Kiri', 'WALL·E',
                         'Capharnaüm', 'Amélie', 'Nausicaä of the Valley of the Wind']

for i in range(len(movie_url_wiki_list)):
    movie_url = movie_url_wiki_list[i]
    title = movie_title_wiki_list[i]
    try:
        # Extracting the plot summary
        wiki_plot = extract_wiki_plot(movie_url)
        assert (wiki_plot is not None and  wiki_plot is not "")
        # Adding wiki plot for the particular movie to movies_df
        add_data_to_df_field(movies_df, movies_df['title'] == title, 'wiki_plot', wiki_plot)
    except:
        print('Plot scraping failed: ', movie_url)

,rank,title,genre,release_date,imdb_synopsis,wiki_plot
30,31,Léon: The Professional,"['Action', 'Crime', 'Drama']",1994,"Léon (Jean Reno) is a hitman (or ""cleaner"" as ...","Léon is an Italian-American hitman (or ""cleane..."


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
31,32,Hara-Kiri,"['Action', 'Drama', 'Mystery']",1962,"""Seppuku,"" or ""Harakiri"" has it is known in th...","Edo, 1630. Tsugumo Hanshirō arrives at the est..."


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
60,61,WALL·E,"['Animation', 'Adventure', 'Family']",2008,A Dystopia in the Future Approximately seven h...,"In the 29th century, rampant consumerism, corp..."


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
76,77,Capharnaüm,['Drama'],2018,"Capernaüm (""Chaos"") tells the story of Zain (Z...","Zain El Hajj, a 12-year-old from the slums of ..."


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
116,117,Amélie,"['Comedy', 'Romance']",2001,Amelie Poulain (Audrey Tautou) is the only chi...,Amélie Poulain is born in June 1974 and brough...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
236,237,Nausicaä of the Valley of the Wind,"['Animation', 'Adventure', 'Fantasy']",1984,An animated fantasy-adventure. Set one thousan...,One thousand years have passed since the Seven...


In [12]:
# Scraping the plot for movie `Hamilton`
title = 'Hamilton'
wiki_url = 'https://en.wikipedia.org/wiki/Hamilton_(2020_film)'
# Extracting the plot summary
movie_html_wiki = read_request(wiki_url)
movie_soup_wiki = BeautifulSoup(movie_html_wiki, 'html.parser')
# Extracting the plot summary
tag  = movie_soup_wiki.select_one('#Synopsis').find_parent('h2')
while tag.name != 'p':
    tag = tag.find_next_sibling()
wiki_plot = tag.get_text(" ", strip=True)
assert (wiki_plot is not None and  wiki_plot is not "")
# Adding wiki plot for the particular movie to movies_df
add_data_to_df_field(movies_df, movies_df['title'] == title, 'wiki_plot', wiki_plot)

# Scraping the plot for movie `Pather_Panchali`
title = 'Pather Panchali'
wiki_url = 'https://en.wikipedia.org/wiki/Pather_Panchali'
# Extracting the plot summary
movie_html_wiki = read_request(wiki_url)
movie_soup_wiki = BeautifulSoup(movie_html_wiki, 'html.parser')
# Extracting the plot summary
tag  = movie_soup_wiki.select_one('#Plot_summary').find_parent('h2').find_next_sibling()
wiki_plot = ''
while tag.name == 'p':
    wiki_plot += tag.text.replace('\n', ' ')
    tag = tag.find_next_sibling()
assert (wiki_plot is not None and  wiki_plot is not "")
# Adding wiki plot for the particular movie to movies_df
add_data_to_df_field(movies_df, movies_df['title'] == title, 'wiki_plot', wiki_plot)

# Scraping the plot for movie `The Battle of Algiers`
title = 'The Battle of Algiers'
wiki_url = 'https://en.wikipedia.org/wiki/The_Battle_of_Algiers'
# Extracting the plot summary
movie_html_wiki = read_request(wiki_url)
movie_soup_wiki = BeautifulSoup(movie_html_wiki, 'html.parser')
# Extracting the plot summary
tag  = movie_soup_wiki.select_one('#Subject').find_parent('h2').find_next_sibling()
wiki_plot = ''
while tag.name == 'p':
    wiki_plot += tag.text.replace('\n', ' ')
    tag = tag.find_next_sibling()

assert (wiki_plot is not None and  wiki_plot is not "")
# Adding wiki plot for the particular movie to movies_df
add_data_to_df_field(movies_df, movies_df['title'] == title, 'wiki_plot', wiki_plot)

# Scraping the plot for `Gangs of Wasseypur`. This movie has been
# divided into two parts because of its original length of over 5 hours
# We will scrape the plots for both parts and combine them.
wiki_plot = ''
title = 'Gangs of Wasseypur'
urls_list = ['https://en.wikipedia.org/wiki/Gangs_of_Wasseypur_%E2%80%93_Part_1', 
             'https://en.wikipedia.org/wiki/Gangs_of_Wasseypur_%E2%80%93_Part_2']
select_par_list = [{'id':'Plot'}, {'id':'Plot.5B13.5D'}]
for i in range(2):
    wiki_url = urls_list[i]
    select_par = select_par_list[i]
    # Extracting the plot summary
    movie_html_wiki = read_request(wiki_url)
    movie_soup_wiki = BeautifulSoup(movie_html_wiki, 'html.parser')
    # Extracting the plot summary
    tag  = movie_soup_wiki.find('span', select_par).find_parent('h2').find_next_sibling()
    while tag.name != 'h2':
        if tag.name == 'p':
            wiki_plot += tag.text.replace('\n', ' ')
        tag = tag.find_next_sibling()

assert (wiki_plot is not None and  wiki_plot is not "")
# Adding wiki plot for the particular movie to movies_df
add_data_to_df_field(movies_df, movies_df['title'] == title, 'wiki_plot', wiki_plot)

,rank,title,genre,release_date,imdb_synopsis,wiki_plot
70,71,Hamilton,"['Biography', 'Drama', 'History']",2020,Act I\nThe orphan Alexander Hamilton leaves hi...,"Divided in two acts, the musical depicts a dra..."


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
85,86,Pather Panchali,['Drama'],1955,(Previous content of this page was removed due...,"In Nischindipur, rural Bengal, in the 1910s, H..."


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
237,238,The Battle of Algiers,"['Drama', 'War']",1966,The opening scene is of a man who has presumab...,The Battle of Algiers reconstructs the events ...


,rank,title,genre,release_date,imdb_synopsis,wiki_plot
223,224,Gangs of Wasseypur,"['Action', 'Crime', 'Drama']",2012,"Prologue\nIn January 2004 , a gang of heavily ...","In January 2004, a gang of heavily armed men s..."


### 5. Inspect the final movies dataframe and save it as a .csv file

We look at our `movies_df` dataframe and inspect it for the missing values. Since there are no more movies with the missing information we go ahead and save the dataframe.

In [13]:
movies_df.head(3)

,rank,title,genre,release_date,imdb_synopsis,wiki_plot
0,1,The Shawshank Redemption,['Drama'],1994,"In 1947, Andy Dufresne ( Tim Robbins ), a bank...","In 1947 Portland, Maine, banker Andy Dufresne ..."
1,2,The Godfather,"['Crime', 'Drama']",1972,"In late summer 1945, guests are gathered for t...","In 1945 New York City, at his daughter Connie'..."
2,3,The Godfather: Part II,"['Crime', 'Drama']",1974,The Godfather Part II presents two parallel st...,The film intercuts between events some time af...


In [14]:
movies_df.isnull().any()

rank             False
title            False
genre            False
release_date     False
imdb_synopsis    False
wiki_plot        False
dtype: bool

In [17]:
# Saving the dataframe to the folder ./data/
movies_df.to_csv('./data/top_250_movies.csv', index=False)